<a href="https://colab.research.google.com/github/Hongchenglong/colab/blob/main/CertifiableBayesianInference/Epsilon_Experiments/MNIST_runner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import sys, os
from pathlib import Path
path = Path(os.getcwd())
sys.path.append(str(path.parent))
sys.path.append('/content/drive/MyDrive/ColabNotebooks/CertifiableBayesianInference')
experiment = '/content/drive/MyDrive/ColabNotebooks/CertifiableBayesianInference/Epsilon_Experiments/'

In [ ]:
# python3 MNIST_runner.py --eps 0.165 --lam 0.25 --rob 1 --opt BBB --gpu 2 &

dict = {'eps': 0.165, 'lam': 0.25, 'rob': 0, 'opt': 'BBB', 'gpu': '2'}

eps = dict['eps']
lam = dict['lam']
rob = dict['rob']
optim = dict['opt']
gpu = dict['gpu']
os.environ['CUDA_VISIBLE_DEVICES'] = gpu

In [ ]:
import BayesKeras
import BayesKeras.optimizers as optimizers

import tensorflow as tf
from tensorflow.keras.models import *
from tensorflow.keras.layers import *

#tf.debugging.set_log_device_placement(True)
#os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train/255.
X_test = X_test/255.
X_train = X_train.astype("float32").reshape(-1, 28*28)
X_test = X_test.astype("float32").reshape(-1, 28*28)

#X_train = X_train[0:10000]
#y_train = y_train[0:10000]

model = Sequential()
model.add(Dense(512, activation="relu", input_shape=(None, None, 28*28)))
model.add(Dense(10, activation="softmax"))

inf = 2
full_covar = False
if(optim == 'VOGN'):
    # was 0.25 for a bit
    inf = 2
    learning_rate = 0.35; decay=0.0
    opt = optimizers.VariationalOnlineGuassNewton()
elif(optim == 'BBB'):
    inf = 2
    learning_rate = 0.45; decay=0.0
    opt = optimizers.BayesByBackprop()
elif(optim == 'SWAG'):
    learning_rate = 0.1; decay=0.0
    opt = optimizers.StochasticWeightAveragingGaussian()
elif(optim == 'SWAG-FC'):
    learning_rate = 0.01; decay=0.0; full_covar=True
    opt = optimizers.StochasticWeightAveragingGaussian()
elif(optim == 'SGD'):
    learning_rate = 1.0; decay=0.0
    opt = optimizers.StochasticGradientDescent()
elif(optim == 'NA'):
    inf = 2
    learning_rate = 0.001; decay=0.0
    opt = optimizers.NoisyAdam()
elif(optim == 'ADAM'):
    learning_rate = 0.00001; decay=0.0
    opt = optimizers.Adam()
elif(optim == 'HMC'):
    learning_rate = 0.075; decay=0.0; inf=250
    linear_schedule = False
    opt = optimizers.HamiltonianMonteCarlo()

# Compile the model to train with Bayesian inference
if(rob == 0):
    loss = tf.keras.losses.SparseCategoricalCrossentropy()
elif(rob != 0):
    loss = BayesKeras.optimizers.losses.robust_crossentropy_loss

bayes_model = opt.compile(model, loss_fn=loss, epochs=20, learning_rate=learning_rate,
                          batch_size=128, linear_schedule=True,
                          decay=decay, robust_train=rob, inflate_prior=inf,
                          burn_in=3, steps=25, b_steps=20, epsilon=eps, rob_lam=lam) #, preload="SGD_FCN_Posterior_1")

This optimizer does not have a default compilation method. Please make sure to call the correct .compile method before use.
BayesKeras: Using implicit prior
(784, 512) 0.050507627227610534
(512, 10) 0.0625
BayesKeras: Using implicit prior
(784, 512) 0.050507627227610534
(512, 10) 0.0625
BayesKeras: Using passed loss_fn as the data likelihood in the KL loss


In [ ]:
#steps was 50
# Train the model on your data
bayes_model.train(X_train, y_train, X_test, y_test)

100%|██████████| 469/469 [00:53<00:00,  8.69it/s]


Epoch 1, loss: 6.759, acc: 0.829, val_loss: 0.266, val_acc: 0.916


100%|██████████| 469/469 [01:21<00:00,  5.72it/s]


Epoch 2, loss: 5.917, acc: 0.934, val_loss: 0.217, val_acc: 0.933


100%|██████████| 469/469 [01:21<00:00,  5.72it/s]


...

Epoch 18, loss: 3.625, acc: 0.975, val_loss: 0.100, val_acc: 0.971


100%|██████████| 469/469 [01:21<00:00,  5.72it/s]


Epoch 19, loss: 3.589, acc: 0.975, val_loss: 0.113, val_acc: 0.969


100%|██████████| 469/469 [00:53<00:00,  8.82it/s]


Epoch 20, loss: 3.550, acc: 0.977, val_loss: 0.104, val_acc: 0.972


In [ ]:
# Save your approxiate Bayesian posterior
bayes_model.save(experiment + "%s_FCN_Posterior_%s"%(optim, rob))

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
